# DAS VERSION - MIX

In [1]:
from datasets import Dataset, concatenate_datasets

import sys
sys.path.append("../..")

import torch
import seaborn as sns
from tqdm import tqdm, trange
import nnsight

from transformers import get_linear_schedule_with_warmup
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from pyvene.models.configuration_intervenable_model import  IntervenableConfig
from pyvene.models.intervenable_base import IntervenableModel
from pyvene.models.interventions import BoundlessRotatedSpaceIntervention
from pyvene.models.basic_utils import set_seed, count_parameters


from utils.prompt_utils import *
from utils.intervention_utils import *
from utils.model_utils import *
from utils.eval_utils import *
from utils.extract_utils import *
from das_utils import *

In [2]:
dataset_names= [
        "antonym",
        "capitalize",
        "english-french",
        "present-past",
        "singular-plural"
    ]

edit_layer = 0
# model_name="/share/projects/engine/weights/llama-hf/llama-7b"
# model_name = "luodian/llama-7b-hf"
model_name = "../flan-llama-7b"
root_data_dir= "../dataset_files"
save_path_root= "../results/DAS/llama-7b"+f"/L{str(edit_layer)}"
intervention_path= None
seed=42
device= "cuda"
test_split= 0.3
n_shots= 10
n_trials= 512
prefixes= {
    "input": "Q:",
    "output": "A:",
    "instructions": ""
}
separators= {
    "input": "\n",
    "output": "\n\n",
    "instructions": ""
}
training_method= "both"
batch_size= 32
gradient_accumulation_steps= 1
epochs= 25
warnup_ratio= 0.1
rotate_lr= 0.001
boundary_lr= 0.01
temperature_start= 50.0
temperature_end= 0.1
evaluate_per_epoch= True


In [3]:
model, tokenizer, model_config = load_gpt_model_and_tokenizer(model_name, device=device)
    
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.pad_token_id = tokenizer.eos_token_id

edit_layer = 12
intervention_path = '/home/local_erictodd/Projects/subspace/models/L12/checkpoints/epoch_24'

assert edit_layer < model_config["n_layers"], f"Edit layer {edit_layer} is out of range for model with {model_config['n_layers']} layers."

if intervention_path is not None:
    print(f"Loading the intervention model from {intervention_path}...")
    intervenable_from = IntervenableModel.load(intervention_path, model=model)
    
    assert len(intervenable_from.interventions.keys()) == 1, "No interventions found in the loaded model."
    
    intervention_key = list(intervenable_from.interventions.keys())[0]
    layer_num = re.findall(r'(?<=layer.)\d+', intervention_key)
    assert layer_num is not None and len(layer_num) == 1, "No layer number found in the loaded model."
    layer_num = int(layer_num[0])
    
    if layer_num != edit_layer:
        print(f"WARNING: The loaded intervention model is for layer {layer_num} but you are running for layer {edit_layer}.")
        print("Loading the weight")
        intervenable_config = simple_boundless_das_position_config(type(model), "block_output", edit_layer)
        intervenable = IntervenableModel(intervenable_config, model)
        intervenable = load_intervention_weight(intervenable, intervenable_from)
    else:
        intervenable = intervenable_from
else:
    intervenable_config = simple_boundless_das_position_config(type(model), "block_output", edit_layer)
    intervenable = IntervenableModel(intervenable_config, model)
    
intervenable.set_device(device)
intervenable.disable_model_gradients()

Loading:  ../flan-llama-7b


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Loading the intervention model from /home/local_erictodd/Projects/subspace/models/L12/checkpoints/epoch_24...


In [4]:
nn_model = nnsight.LanguageModel(model_name, device_map='cuda:0', dispatch=True)
for p in nn_model.parameters():
    p.requires_grad = False
learned_proj = BoundlessRotatedSpaceIntervention(embed_dim=nn_model.config.hidden_size)
learned_proj.load_state_dict(torch.load('../results/proj_l12_state_dict.pt'))
learned_proj.to('cuda')

BoundlessRotatedSpaceIntervention(
  (rotate_layer): ParametrizedRotateLayer(
    (parametrizations): ModuleDict(
      (weight): ParametrizationList(
        (0): _Orthogonal()
      )
    )
  )
)

In [8]:
inputs = torch.load('match_das_results/inputs.pt')

In [19]:
# p1 = 'Q: control\nA:'
# p2 = 


# nn_model.tokenizer.decode(inputs['base_input_ids'][2])

'<s> Q: manipulate\nA: manip</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>'

In [3]:
results = dict()


# Load Model & Tokenizer
print("Loading Model")
# model, tokenizer, model_config = load_gpt_model_and_tokenizer(model_name, device=device)

# # tokenizer.pad_token = tokenizer.eos_token
# # tokenizer.padding_side = "right"
# tokenizer.pad_token_id = tokenizer.eos_token_id

# assert edit_layer < model_config["n_layers"], f"Edit layer {edit_layer} is out of range for model with {model_config['n_layers']} layers."

# if intervention_path is not None:
#     print(f"Loading the intervention model from {intervention_path}...")
#     intervenable = IntervenableModel.load(intervention_path, model=model)
# else:
#     intervenable_config = simple_boundless_das_position_config(type(model), "block_output", edit_layer)
#     intervenable = IntervenableModel(intervenable_config, model)
    
# intervenable.set_device(device)
# intervenable.disable_model_gradients()


nn_model = nnsight.LanguageModel(model_name, device_map='cuda:0', dispatch=True)
# nn_model = nnsight.LanguageModel(model, device_map='cuda:0', tokenizer=tokenizer)
for p in nn_model.parameters():
    p.requires_grad = False
learnable_proj = BoundlessRotatedSpaceIntervention(embed_dim=model.config.hidden_size)
learnable_proj.to('cuda')
    
def vanilla_collate_fn(batch):
    input_ids, labels = tuple([data_pair[key] for data_pair in batch] for key in ("input_ids", "labels"))
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
    
    return dict(
        input_ids=input_ids,
        labels=labels,
        attention_mask=input_ids.ne(tokenizer.pad_token_id),
    )
    
def intervention_collate_fn(batch):
    base_input_ids, base_labels, source_input_ids, source_labels, source_predictive_token_idxs, predictive_token_idxs = tuple(
        [data_pair[key] for data_pair in batch] for key in 
        ('base_input_ids', 'base_labels', 'source_input_ids', 'source_labels', 'source_predictive_token_idxs', 'predictive_token_idxs')
    )
    
    base_input_ids = torch.nn.utils.rnn.pad_sequence(
        base_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )
    
    source_input_ids = torch.nn.utils.rnn.pad_sequence(
        source_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )
    
    base_labels = torch.nn.utils.rnn.pad_sequence(base_labels, batch_first=True, padding_value=IGNORE_INDEX)
    source_labels = torch.nn.utils.rnn.pad_sequence(source_labels, batch_first=True, padding_value=IGNORE_INDEX)
    
    source_predictive_token_idxs = torch.LongTensor(source_predictive_token_idxs)
    predictive_token_idxs = torch.LongTensor(predictive_token_idxs)
    
    return dict(
        base_input_ids=base_input_ids,
        base_labels=base_labels,
        base_attention_mask=base_input_ids.ne(tokenizer.pad_token_id),
        source_input_ids=source_input_ids,
        source_labels=source_labels,
        source_attention_mask=source_input_ids.ne(tokenizer.pad_token_id),
        predictive_token_idxs=predictive_token_idxs,
        source_predictive_token_idxs=source_predictive_token_idxs
    )

def calculate_loss(logits, labels):
    shift_logits = logits[..., :, :].contiguous()
    shift_labels = labels[..., :].contiguous()
    # Flatten the tokens
    loss_fct = CrossEntropyLoss()
    shift_logits = shift_logits.view(-1, intervenable.model_config.vocab_size)
    shift_labels = shift_labels.view(-1)
    # Enable model parallelism
    shift_labels = shift_labels.to(shift_logits.device)
    loss = loss_fct(shift_logits, shift_labels)
    
    for k, v in intervenable.interventions.items():
        boundary_loss = 1. * v[0].intervention_boundaries.sum()
    loss += boundary_loss
    
    return loss

# Load the dataset
print("Loading Dataset")
set_seed(seed)
datasets = [load_dataset(dataset_name, root_data_dir=root_data_dir, test_size=test_split, seed=seed) for dataset_name in dataset_names]

if not os.path.exists(save_path_root):
    os.makedirs(save_path_root)
    
print("Processing Dataloaders")

eval_no_intervention_dataloader = process_mixed_dataloader(datasets, model_config, tokenizer, batch_size, n_shots, "valid", prefixes, separators, intervention_collate_fn, ablation_method="zero_shot", draw_source_from_split=True)
if training_method == "both":
    
    all_datasets = []
    for method in ["zero_shot", "noninformative"]:
        for dataset in datasets:
            all_datasets.append(process_dataset(dataset, model_config, tokenizer, n_shots, "train", prefixes, separators, n_trials=n_trials, ablation_method=method, draw_source_from_split=False))
    train_dataset = concatenate_datasets(all_datasets)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=intervention_collate_fn)
else:
    train_dataloader = process_mixed_dataloader(datasets, model_config, tokenizer, batch_size, n_shots, "train", prefixes, separators, intervention_collate_fn, n_trials=n_trials, ablation_method=training_method, draw_source_from_split=False)
    
fs_eval_dataloader = process_mixed_dataloader(datasets, model_config, tokenizer, batch_size, n_shots, "valid", prefixes, separators, intervention_collate_fn, ablation_method="noninformative", draw_source_from_split=True)
zs_eval_dataloader = process_mixed_dataloader(datasets, model_config, tokenizer, batch_size, n_shots, "valid", prefixes, separators, intervention_collate_fn, ablation_method="zero_shot", draw_source_from_split=True)

print(f"Evaluating the model {n_shots}-shots without intervention...")
eval_accuracy = evaluate(intervenable, eval_no_intervention_dataloader, device=model.device, intervene=False, corrupt=False)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {eval_accuracy}")
results['prealign_val_task_accuracy'] = eval_accuracy


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading Model
Loading:  luodian/llama-7b-hf


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Dataset
Processing Dataloaders
Evaluating the model 10-shots without intervention...


100%|██████████| 24/24 [00:15<00:00,  1.52it/s]


[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: 0.75


In [4]:

t_total = int(len(train_dataloader) * epochs)
warm_up_steps = 0.1 * t_total
optimizer_params = []

for k, v in intervenable.interventions.items():
    optimizer_params += [{'params': v[0].rotate_layer.parameters()}]
    optimizer_params += [{'params': v[0].intervention_boundaries, 'lr': boundary_lr}]

optimizer = torch.optim.Adam(
    optimizer_params,
    lr=rotate_lr,
)

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warm_up_steps,
    num_training_steps=t_total
)

target_total_step = len(train_dataloader) * epochs

temperature_schedule = torch.linspace(
    temperature_start, temperature_end, target_total_step
).to(torch.bfloat16).to(device)

total_step = 0
intervenable.set_temperature(temperature_schedule[total_step])

intervenable.model.train() # train enables drop-off but no grads
print("llama trainable parameters: ", count_parameters(intervenable.model))
print("intervention trainable parameters: ", intervenable.count_parameters())

train_iterator = trange(
    0, int(epochs), desc="Epoch"
)

training_log_dicts = None

llama trainable parameters:  0
intervention trainable parameters:  16777218


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

In [5]:
if intervention_path is None:
    
    os.makedirs(os.path.join(save_path_root, "checkpoints"), exist_ok=True)
    
    training_log_dicts = []
            
    for epoch in train_iterator:
        
        log_dicts = []
        ckpt_path = os.path.join(save_path_root, "checkpoints", f"epoch_{epoch}")
        
        epoch_iterator = tqdm(
            train_dataloader, desc=f"Epoch: {epoch}", position=0, leave=True
        )
        
        for step, inputs in enumerate(epoch_iterator):
            for k, v in inputs.items():
                if v is not None and isinstance(v, torch.Tensor):
                    inputs[k] = v.to(device)
            b_s = inputs["base_input_ids"].shape[0]

            source2base = ([[[idx] for idx in inputs["source_predictive_token_idxs"].tolist()]], [[[idx] for idx in inputs["predictive_token_idxs"].tolist()]])
            
            _, counterfactual_outputs = intervenable(
                {"input_ids": inputs["base_input_ids"], "attention_mask": inputs["base_attention_mask"]},
                [{"input_ids": inputs["source_input_ids"], "attention_mask": inputs["source_attention_mask"]}],
                {"sources->base": source2base}
            )
            
            eval_metrics = compute_metrics(
                [counterfactual_outputs.logits], [inputs['base_labels']]
            )
            
            loss = calculate_loss(
                counterfactual_outputs.logits, inputs["base_labels"]
            )
            loss_str = round(loss.item(), 2)
            
            log_dict = {'loss': loss_str, 'acc': eval_metrics["accuracy"], 'sparsity': compute_rotation_mask_sparsity(intervenable)}
            epoch_iterator.set_postfix(log_dict)
            
            log_dicts.append(log_dict)
            
            if gradient_accumulation_steps > 1:
                loss = loss / gradient_accumulation_steps
                
            loss.backward()
            # if total_step % gradient_accumulation_steps == 0:
            #     if not (gradient_accumulation_steps > 1 and total_step == 0):
            #         optimizer.step()
            #         scheduler.step()
            #         intervenable.set_zero_grad()
            #         intervenable.set_temperature(temperature_schedule[total_step])
                    
            # total_step += 1
            break
        break

        
#         ave_loss = round(sum([log_dict['loss'] for log_dict in log_dicts])/len(log_dicts), 4)
#         ave_acc = round(sum([log_dict['acc'] for log_dict in log_dicts])/len(log_dicts), 4)
#         ave_sparsity = round(sum([log_dict['sparsity'] for log_dict in log_dicts])/len(log_dicts), 4) 
        
#         epoch_training_log = {'loss': ave_loss, 'acc': ave_acc, 'sparsity': ave_sparsity}
#         print("Epoch " + str(epoch) + " finished! Training loss: " + str(ave_loss) + ", training acc: " + str(ave_acc) + ", sparsity: " + str(ave_sparsity))
        
#         if evaluate_per_epoch:
            
#             fs_shuffled_acc = evaluate(intervenable, fs_eval_dataloader, device=model.device, intervene=True)
#             epoch_training_log['fs_shuffled_with_intervention_accuracy'] = fs_shuffled_acc
            
#             zs_intervention_acc = evaluate(intervenable, zs_eval_dataloader, device=model.device, intervene=True)
#             epoch_training_log['zs_with_intervention_accuracy'] = zs_intervention_acc
            
#             print("Few-shot shuffled with intervention accuracy: " + str(epoch_training_log['fs_shuffled_with_intervention_accuracy']))
#             print("Zero-shot with intervention accuracy: " + str(epoch_training_log['zs_with_intervention_accuracy']))
        
#         intervenable.save(ckpt_path)
#         training_log_dicts.append(epoch_training_log)
    
# print("Evaluation the model with intervention...")

# fs_shuffled_acc = evaluate(intervenable, fs_eval_dataloader, device=model.device, intervene=True)
# results['fs_shuffled_with_intervention_accuracy'] = fs_shuffled_acc

# fs_shuffled_no_intervention_acc = evaluate(intervenable, fs_eval_dataloader, device=model.device, intervene=False, corrupt=True)
# results['fs_shuffled_no_intervention_accuracy'] = fs_shuffled_no_intervention_acc

# zs_intervention_acc = evaluate(intervenable, zs_eval_dataloader, device=model.device, intervene=True)
# results['zs_with_intervention_accuracy'] = zs_intervention_acc
    
# zs_no_intervention_acc = evaluate(intervenable, zs_eval_dataloader, device=model.device, intervene=False, corrupt=True)
# results['zs_no_intervention_accuracy'] = zs_no_intervention_acc

# print("Few-shot shuffled with intervention accuracy: " + str(results['fs_shuffled_with_intervention_accuracy']))
# print("Few-shot shuffled no intervention accuracy: " + str(results['fs_shuffled_no_intervention_accuracy']))
# print("Zero-shot with intervention accuracy: " + str(results['zs_with_intervention_accuracy']))
# print("Zero-shot no intervention accuracy: " + str(results['zs_no_intervention_accuracy']))

# print("Saving results...")

# with open(f"{save_path_root}/args.json", "w") as f:
#     json.dump(vars(args), f, indent=4)
#     f.close()
    
# with open(f"{save_path_root}/results.json", "w") as f:
#     json.dump(results, f, indent=4)
    
# if training_log_dicts is not None:
#     with open(f"{save_path_root}/training_log.json", "w") as f:
#         json.dump(training_log_dicts, f, indent=4)
#         f.close()
        
# intervenable.save(f"{save_path_root}/intervention_model")
# print("Done!")


Epoch:   0%|          | 0/25 [00:04<?, ?it/s]


## NNSIGHT MODEL TESTING

In [1]:
from datasets import Dataset, concatenate_datasets

import sys
sys.path.append("../..")

import torch
import seaborn as sns
from tqdm import tqdm, trange
import nnsight

from transformers import get_linear_schedule_with_warmup
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

# from pyvene.models.configuration_intervenable_model import  IntervenableConfig, RepresentationConfig
# from pyvene.models.intervenable_base import IntervenableModel
from pyvene.models.interventions import BoundlessRotatedSpaceIntervention
# from pyvene.models.basic_utils import set_seed, count_parameters


from utils.prompt_utils import *
from utils.intervention_utils import *
from utils.model_utils import *
from utils.eval_utils import *
from utils.extract_utils import *
from das_utils import *

dataset_names= [
        "antonym",
        "capitalize",
        "english-french",
        "present-past",
        "singular-plural"
    ]

edit_layer = 0
# model_name="/share/projects/engine/weights/llama-hf/llama-7b"
model_name = "luodian/llama-7b-hf"
root_data_dir= "../dataset_files"
save_path_root= "../results/DAS/llama-7b"+f"/L{str(edit_layer)}"
intervention_path= None
seed=42
device= "cuda"
test_split= 0.3
n_shots= 10
n_trials= 512
prefixes= {
    "input": "Q:",
    "output": "A:",
    "instructions": ""
}
separators= {
    "input": "\n",
    "output": "\n\n",
    "instructions": ""
}
training_method= "both"
batch_size= 32
gradient_accumulation_steps= 1
epochs= 25
warnup_ratio= 0.1
rotate_lr= 0.001
boundary_lr= 0.01
temperature_start= 50.0
temperature_end= 0.1
evaluate_per_epoch= True


In [2]:
class RotateLayer(torch.nn.Module):
    """A linear transformation with orthogonal initialization."""

    def __init__(self, n, init_orth=True):
        super().__init__()
        weight = torch.empty(n, n)
        if init_orth:
            torch.nn.init.orthogonal_(weight)
        self.weight = torch.nn.Parameter(weight, requires_grad=True)

    def forward(self, x):
        return torch.matmul(x.to(self.weight.dtype), self.weight)
    
    
class PyveneIntervention(torch.nn.Module):
    """Intervention in the rotated space with boundary mask."""

    def __init__(self, embed_dim, steps, init_value=100.0, random_mask=False):
        super(PyveneIntervention, self).__init__()
        self.embed_dim = embed_dim

        rotate_layer = RotateLayer(self.embed_dim)
        self.rotate_layer = torch.nn.utils.parametrizations.orthogonal(rotate_layer)
        
        if not random_mask:
            self.mask = torch.nn.Parameter(torch.tensor([init_value] * self.embed_dim), requires_grad=True) # [100.0]
        else:
            self.mask = torch.nn.Parameter((-100 - 100) * torch.rand(self.embed_dim) + 100) # uniform on [-100, 100]
        
        self.temp_range = torch.linspace(torch.tensor(50.0), torch.tensor(0.10), steps+1).to('cuda')
        self.curr_step = 0
        self.temperature = torch.nn.Parameter(torch.tensor(50.0)).to('cuda')

    def get_temperature(self):
        return self.temperature

    def update_temp(self):
        self.curr_step += 1
        self.temperature.data = self.temp_range[self.curr_step]
    
    def effective_subspace_dim(self):
        return sum(torch.sigmoid(self.mask / self.temperature)).item()

    def forward(self, source, base):

        rotated_base = self.rotate_layer(base)
        rotated_source = self.rotate_layer(source)
        # get boundary mask between 0 and 1 from sigmoid
        boundary_mask = torch.sigmoid(self.mask / self.temperature)

        boundary_mask = (
            # torch.ones_like(base, device=base.device)[:,0,0].unsqueeze(dim=-1) * boundary_mask
            # torch.ones(base.shape[0], device=base.device).unsqueeze(dim=-1) * boundary_mask
            torch.ones_like(base)[:,0].unsqueeze(dim=-1).to(base.device) * boundary_mask
        )
        boundary_mask = boundary_mask.to(rotated_base.dtype)
        # interchange
        rotated_output = (
            1.0 - boundary_mask
        ) * rotated_base + boundary_mask * rotated_source
        # inverse output
        output = torch.matmul(rotated_output, self.rotate_layer.weight.T)
        return output.to(base.dtype).squeeze()

    def __str__(self):
        return f"PyveneIntervention()"

In [3]:
inputs = torch.load('match_das_results/inputs.pt')

In [4]:
nn_model = nnsight.LanguageModel(model_name, device_map='cuda:0', dispatch=True)
for p in nn_model.parameters():
    p.requires_grad = False
learned_proj = PyveneIntervention(embed_dim=nn_model.config.hidden_size, steps=10000)
# learned_proj = BoundlessRotatedSpaceIntervention(embed_dim=nn_model.config.hidden_size)
# learned_proj.load_state_dict(torch.load('../results/proj_l12_state_dict.pt'))
learned_proj.to('cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PyveneIntervention(
  (rotate_layer): ParametrizedRotateLayer(
    (parametrizations): ModuleDict(
      (weight): ParametrizationList(
        (0): _Orthogonal()
      )
    )
  )
)

In [5]:
import inspect
def intervene_llama_DAS_batch(batch, layer, model, ii_proj):


    # batch_size = len(batch['base_input_ids'])
    batch_size = 8
    all_inds = torch.arange(batch_size)
        
    base_prompt, source_prompt = batch['base_input_ids'][:batch_size], batch['source_input_ids'][:batch_size]
    base_intervention_token_idx, source_intervention_token_idx = batch['predictive_token_idxs'][:batch_size], batch['source_predictive_token_idxs'][:batch_size]

    with model.trace(validate=False) as tracer:

        with tracer.invoke(base_prompt, scan=False):
            base = model.model.layers[layer].output[0]
        
        with tracer.invoke(source_prompt, scan=False):
            source = model.model.layers[layer].output[0]

        # intervention
        with tracer.invoke(base_prompt, scan=False):
            B = base[all_inds,base_intervention_token_idx,:]
            S = source[all_inds,source_intervention_token_idx,:]

            mixed_out = ii_proj(S, B)
            model.model.layers[layer].output[0][all_inds,base_intervention_token_idx,:] = mixed_out
            del base, source, B,S
        save_out = model.output.save()
    
    
    # print(b_shape)
    output_logits = save_out.value.logits[1::3]
    del save_out
    return output_logits

In [6]:
# attn_in = []
# attn_out = []
# with nn_model.trace() as tracer:
#     with tracer.invoke('hello world base'):
#         for i in range(nn_model.config.num_hidden_layers):
#             source_attn_in = nn_model.model.layers[i].self_attn.o_proj.input.save()
            
#     with tracer.invoke('hello world source'):
#         for i in range(nn_model.config.num_hidden_layers):
#             base_attn_in = nn_model.model.layers[i].self_attn.o_proj.input[0][0].clone().save()
#             nn_model.model.layers[i].self_attn.o_proj.input[0][0] = base_attn_in[0][0] + source_attn_in[0][0]

#             post_interv = nn_model.model.layers[i].self_attn.o_proj.input[0][0].clone().save()

#             # hidden_states_post = nn_model.model.layers[i].self_attn.o_proj.input.save()
            

# # print(attn_out)

In [7]:
outs = intervene_llama_DAS_batch(inputs, 12, nn_model, learned_proj)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [8]:
outs.shape

torch.Size([8, 132, 32000])

In [38]:
inputs['predictive_token_idxs'][:3]

nn_model.tokenizer.decode(inputs['source_input_ids'][6].squeeze())

'<s> Q: control\nA: controlled\n\nQ: discover\nA: discovered\n\nQ: investigate\nA: investigated\n\nQ: join\nA: joined\n\nQ: form\nA: formed\n\nQ: compete\nA: competed\n\nQ: warn\nA: warned\n\nQ: play\nA: played\n\nQ: change\nA: changed\n\nQ: accelerate\nA: accelerated\n\nQ: let\nA:</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>'

In [26]:
# nn_model.tokenizer.decode(outs[torch.arange(3),inputs['predictive_token_idxs'][:3]].softmax(-1).argmax(-1))
outs.shape

torch.Size([3, 132, 32000])

In [7]:
# counterfactual_outputs.shape
# del counterfactual_outputs
# torch.cuda.empty_cache()

# Recreate

In [106]:
def intervene_llama_DAS(batch, layer, model, ii_proj):

    batch_size = len(batch['base_input_ids'])
    # batch_size = 2

    with model.trace() as tracer:

        for i  in range(batch_size):
            base_prompt, source_prompt = batch['base_input_ids'][i], batch['source_input_ids'][i]
            base_intervention_token_idx, source_intervention_token_idx = batch['predictive_token_idxs'][i].item(), batch['source_predictive_token_idxs'][i].item()

            with tracer.invoke(base_prompt):
                base = model.model.layers[layer].output[0]
                # print(base)
                
            
            with tracer.invoke(source_prompt):
                source = model.model.layers[layer].output[0]
        
            B = base[:,base_intervention_token_idx,:].detach()
            S = source[:,source_intervention_token_idx,:].detach()

            # intervention
            with tracer.invoke(base_prompt):
                # pass
                
                X = ii_proj(B,S)
                # print((B + S).sum())
                # model.model.layers[layer].output[0][:,base_intervention_token_idx,:] = 
                # model.model.layers[layer].output[0].token[base_intervention_token_idx] = ii_proj(base[:,base_intervention_token_idx,:], source[:,source_intervention_token_idx,:])
            
        save_out = model.output.save()
    
    # print(base, base.value)
    
    return save_out.value

In [8]:
# def intervene_llama_DAS_batch(batch, layer, model, ii_proj):

#     batch_size = len(batch['base_input_ids'])
#     all_inds = torch.arange(batch_size)
#     # batch_size = 2

#     with model.trace() as tracer:

#         # for i  in range(batch_size):
#         base_prompt, source_prompt = batch['base_input_ids'], batch['source_input_ids']
#         base_intervention_token_idx, source_intervention_token_idx = batch['predictive_token_idxs'], batch['source_predictive_token_idxs']

#         with tracer.invoke(base_prompt):
#             base = model.model.layers[layer].output[0]
        
#         with tracer.invoke(source_prompt):
#             source = model.model.layers[layer].output[0]

#         B = base[all_inds,base_intervention_token_idx,:].detach()
#         S = source[all_inds,source_intervention_token_idx,:].detach()
#         print(B.shape, S.shape)
    
#         # intervention
#         with tracer.invoke(base_prompt):
#             # base = model.model.layers[layer].output[0]
#             # print(base.shape, base_intervention_token_idx, source.shape, source_intervention_token_idx)
            
#             model.model.layers[layer].output[0][all_inds,base_intervention_token_idx,:] = ii_proj(B, S)
#                 # model.model.layers[layer].output[0].token[base_intervention_token_idx] = ii_proj(base[:,base_intervention_token_idx,:], source[:,source_intervention_token_idx,:])
            
#         save_out = model.output.save()
    
#     return save_out.value

In [ ]:
# def intervene_llama_DAS(batch, layer, model, ii_proj):

#     batch_size = len(batch['base_input_ids'])
#     # batch_size = 2

#     with model.trace(validate=False) as tracer:

#         for i  in range(batch_size):
#             base_prompt, source_prompt = batch['base_input_ids'][i], batch['source_input_ids'][i]
#             base_intervention_token_idx, source_intervention_token_idx = batch['predictive_token_idxs'][i].item(), batch['source_predictive_token_idxs'][i].item()

#             with tracer.invoke(base_prompt, scan=False):
#                 base = model.model.layers[layer].output
#                 print(base.shape)
                
            
#             with tracer.invoke(source_prompt, scan=False):
#                 source = model.model.layers[layer].output
        
#             # intervention
#             with tracer.invoke(base_prompt, scan=False):
#                 model.model.layers[layer].output[:,base_intervention_token_idx,:] = ii_proj(base[:,base_intervention_token_idx,:], source[:,source_intervention_token_idx,:])
#                 # model.model.layers[layer].output[0].token[base_intervention_token_idx] = ii_proj(base[:,base_intervention_token_idx,:], source[:,source_intervention_token_idx,:])
            
#         save_out = model.output.save()
    
#     # print(base.value[0].shape)
    
#     return save_out.value

In [22]:
attn_in = []
attn_out = []
with nn_model.trace() as tracer:
    with tracer.invoke('hello world base'):
        for i in range(nn_model.config.num_hidden_layers):
            source_attn_in = nn_model.model.layers[i].self_attn.o_proj.input.save()
            
    with tracer.invoke('hello world source'):
        for i in range(nn_model.config.num_hidden_layers):
            base_attn_in = nn_model.model.layers[i].self_attn.o_proj.input[0][0].clone().save()
            nn_model.model.layers[i].self_attn.o_proj.input[0][0] = base_attn_in[0][0] + source_attn_in[0][0]

            post_interv = nn_model.model.layers[i].self_attn.o_proj.input[0][0].clone().save()

            # hidden_states_post = nn_model.model.layers[i].self_attn.o_proj.input.save()
            

# print(base_attn_in, post_interv)

TypeError: 'tuple' object does not support item assignment

In [ ]:
with model.trace():

    

In [27]:
source_attn_in

LanguageModelProxy (argument_32): ((FakeTensor(..., device='cuda:0', size=(1, 4, 4096)),), {})